# Querying from Mongo DataBase

In this notebook, we'll make querys to our Mongo database and create a pandas dataframe. 

-----

**First things first, let's import some things...**

In [3]:
import json
import pandas as pd
from pymongo import MongoClient

<br>

**Creating a Mongo Client**

In [4]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client['pbmnet']
curriculos = db.curriculos

<br>

**Querying...**

## Collecting 'Author-Paper'

Dataframe to create a Network

In [5]:
# curriculos.find_one({"PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.AUTORES.@NOME-PARA-CITACAO":"BONOTTO, E.M."})
d = curriculos.find_one({},{'_id':0,'@NUMERO-IDENTIFICADOR':1,'DADOS-GERAIS.@NOME-COMPLETO': 1,
                            'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@TITULO-DO-ARTIGO': 1,
                           'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@ANO-DO-ARTIGO':1})

In [8]:
d['DADOS-GERAIS']['@NOME-COMPLETO']

'Manuel Francisco Zuloeta Jimenez'

In [20]:
unwind = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@NOME-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@CODIGO-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@PAIS':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@UF':1,
        'DADOS-GERAIS.@PAIS-DE-NACIONALIDADE': 1,
        'DADOS-GERAIS.@NOME-EM-CITACOES-BIBLIOGRAFICAS': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@TITULO-DO-ARTIGO': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@ANO-DO-ARTIGO':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DETALHAMENTO-DO-ARTIGO.@TITULO-DO-PERIODICO-OU-REVISTA':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.PALAVRAS-CHAVE':1
                                       }},
                            {'$unwind': '$PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO'},
                          ]))

In [21]:
def Unest_Titulo_Artigo(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'].strip()

def Unest_Endereco_Nome(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@NOME-INSTITUICAO-EMPRESA']
    except:
        value = None
    return value
def Unest_Endereco_UF_Insituicao(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@UF']
    except:
        value = None
    return value
def Unest_Endereco_Pais_Insituicao(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@PAIS']
    except:
        value = None
    return value

def Unest_Endereco_Codigo(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@CODIGO-INSTITUICAO-EMPRESA']
    except:
        value = None
    return value

def Unest_Nacionalidade(cell):
    try:
        value = cell['@PAIS-DE-NACIONALIDADE']
    except:
        value = None
    return value

def Unest_Ano_Artigo(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO'].strip()

def Unest_Palavras_Chave(cell):
    try:
        value = cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['PALAVRAS-CHAVE']
    except:
        value = None
    return value

def Unest_Journal(cell):
    try:
        value = cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DETALHAMENTO-DO-ARTIGO']['@TITULO-DO-PERIODICO-OU-REVISTA'].strip()
    except:
        value = None
    return value

In [22]:
unwind[0]

{'_id': ObjectId('5d6551dfc0e1f77ce11401ba'),
 '@NUMERO-IDENTIFICADOR': '6115398975075240',
 'DADOS-GERAIS': {'@NOME-COMPLETO': 'Manuel Francisco Zuloeta Jimenez',
  '@NOME-EM-CITACOES-BIBLIOGRAFICAS': 'M. Z. Jimenez;M. Z. JIMENEZ;JIMENEZ, M. Z.;JIMENEZ, M.Z.',
  '@PAIS-DE-NACIONALIDADE': 'Peru',
  'ENDERECO': {'ENDERECO-PROFISSIONAL': {'@CODIGO-INSTITUICAO-EMPRESA': '198100000000',
    '@NOME-INSTITUICAO-EMPRESA': 'Universidade Tecnológica Federal do Paraná',
    '@PAIS': 'Brasil',
    '@UF': 'PR'}}},
 'PRODUCAO-BIBLIOGRAFICA': {'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Weak almost periodic motions, minimality and stability in impulsive semidynamical systems',
     '@ANO-DO-ARTIGO': '2014'},
    'DETALHAMENTO-DO-ARTIGO': {'@TITULO-DO-PERIODICO-OU-REVISTA': 'Journal of Differential Equations (Print)'},
    'PALAVRAS-CHAVE': {'@PALAVRA-CHAVE-1': 'Sistemas semidinâmicos Impulsivos',
     '@PALAVRA-CHAVE-2': 'Weak almost periodic motions

In [23]:
def Separar_Palavras_Chave(cell):
    if cell == None:
        return None
    return '|'.join([i for i in cell.values()])

In [24]:
df = pd.DataFrame(unwind)

df['NOME-COMPLETO']     = df["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-COMPLETO'].strip())
df['NOME-INSTITUICAO']  = df["DADOS-GERAIS"].apply(Unest_Endereco_Nome)
df['UF-INSTITUICAO']    = df["DADOS-GERAIS"].apply(Unest_Endereco_UF_Insituicao)
df['PAIS-INSTITUICAO']  = df["DADOS-GERAIS"].apply(Unest_Endereco_Pais_Insituicao)
df['PAIS']              = df["DADOS-GERAIS"].apply(Unest_Nacionalidade)
df['NOME-CITACOES']     = df["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-EM-CITACOES-BIBLIOGRAFICAS'].strip())
df['CODIGO-INSTITUICAO']= df["DADOS-GERAIS"].apply(Unest_Endereco_Codigo)
df['TITULO-DO-ARTIGO']  = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Titulo_Artigo)
df['ANO-DO-ARTIGO']     = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Ano_Artigo)
df['JOURNAL']           = df['PRODUCAO-BIBLIOGRAFICA'].apply(Unest_Journal)
df['PALAVRAS-CHAVE-ARTIGO'] = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Palavras_Chave)
df['PALAVRAS-CHAVE-ARTIGO'] = df['PALAVRAS-CHAVE-ARTIGO'].apply(Separar_Palavras_Chave)

df.drop(labels=['PRODUCAO-BIBLIOGRAFICA','DADOS-GERAIS'],axis=1,inplace=True)

In [25]:
df.to_csv("./views/Author_Paper.csv",sep=';',encoding='latin-1')

<br>

## Collecting 'Dados Gerais'

In [54]:
unwind2 = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@NOME-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@CODIGO-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@PAIS':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@UF':1,
        'DADOS-GERAIS.@PAIS-DE-NACIONALIDADE': 1,
        'DADOS-GERAIS.@NOME-EM-CITACOES-BIBLIOGRAFICAS': 1
                            }}      ]))

In [55]:
unwind2[0]

{'_id': ObjectId('5d6551dfc0e1f77ce11401ba'),
 '@NUMERO-IDENTIFICADOR': '6115398975075240',
 'DADOS-GERAIS': {'@NOME-COMPLETO': 'Manuel Francisco Zuloeta Jimenez',
  '@NOME-EM-CITACOES-BIBLIOGRAFICAS': 'M. Z. Jimenez;M. Z. JIMENEZ;JIMENEZ, M. Z.;JIMENEZ, M.Z.',
  '@PAIS-DE-NACIONALIDADE': 'Peru',
  'ENDERECO': {'ENDERECO-PROFISSIONAL': {'@CODIGO-INSTITUICAO-EMPRESA': '198100000000',
    '@NOME-INSTITUICAO-EMPRESA': 'Universidade Tecnológica Federal do Paraná',
    '@PAIS': 'Brasil',
    '@UF': 'PR'}}}}

In [59]:
df2 = pd.DataFrame(unwind2)

df2['NOME-COMPLETO']     = df2["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-COMPLETO'].strip())
df2['NOME-INSTITUICAO']  = df2["DADOS-GERAIS"].apply(Unest_Endereco_Nome)
df2['UF-INSTITUICAO']    = df2["DADOS-GERAIS"].apply(Unest_Endereco_UF_Insituicao)
df2['PAIS-INSTITUICAO']  = df2["DADOS-GERAIS"].apply(Unest_Endereco_Pais_Insituicao)
df2['PAIS']              = df2["DADOS-GERAIS"].apply(Unest_Nacionalidade)
df2['NOME-CITACOES']     = df2["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-EM-CITACOES-BIBLIOGRAFICAS'].strip())
df2['CODIGO-INSTITUICAO']= df2["DADOS-GERAIS"].apply(Unest_Endereco_Codigo)

df2.drop(labels='DADOS-GERAIS',axis=1,inplace=True)

In [60]:
df2.shape[0]

11423

In [62]:
df2.to_csv('views/Dados_gerais.csv', sep=';', encoding='latin-1')

<br>

## Collecting 'Papers'

In [64]:
unwind3 = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@TITULO-DO-ARTIGO': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@ANO-DO-ARTIGO':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DETALHAMENTO-DO-ARTIGO.@TITULO-DO-PERIODICO-OU-REVISTA':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.PALAVRAS-CHAVE':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.AUTORES': 1
                                       }},
                            {'$unwind': '$PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO'},
                          ]))

In [65]:
unwind3[0]

{'_id': ObjectId('5d6551dfc0e1f77ce11401ba'),
 '@NUMERO-IDENTIFICADOR': '6115398975075240',
 'DADOS-GERAIS': {'@NOME-COMPLETO': 'Manuel Francisco Zuloeta Jimenez'},
 'PRODUCAO-BIBLIOGRAFICA': {'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Weak almost periodic motions, minimality and stability in impulsive semidynamical systems',
     '@ANO-DO-ARTIGO': '2014'},
    'DETALHAMENTO-DO-ARTIGO': {'@TITULO-DO-PERIODICO-OU-REVISTA': 'Journal of Differential Equations (Print)'},
    'AUTORES': [{'@NOME-COMPLETO-DO-AUTOR': 'BONOTTO, E.M.',
      '@NOME-PARA-CITACAO': 'BONOTTO, E.M.',
      '@ORDEM-DE-AUTORIA': '1',
      '@NRO-ID-CNPQ': ''},
     {'@NOME-COMPLETO-DO-AUTOR': 'Manuel Francisco Zuloeta Jimenez',
      '@NOME-PARA-CITACAO': 'JIMENEZ, M.Z.',
      '@ORDEM-DE-AUTORIA': '2',
      '@NRO-ID-CNPQ': ''}],
    'PALAVRAS-CHAVE': {'@PALAVRA-CHAVE-1': 'Sistemas semidinâmicos Impulsivos',
     '@PALAVRA-CHAVE-2': 'Weak almost periodic motions',
 

In [66]:
df3 = pd.DataFrame(unwind3)

In [70]:
def Unest_Coautores(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['AUTORES']

In [75]:
df3['NOME-COMPLETO']     = df3["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-COMPLETO'].strip())
df3['TITULO-DO-ARTIGO']  = df3["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Titulo_Artigo)
df3['ANO-DO-ARTIGO']     = df3["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Ano_Artigo)
df3['JOURNAL']           = df3['PRODUCAO-BIBLIOGRAFICA'].apply(Unest_Journal)
df3['PALAVRAS-CHAVE-ARTIGO'] = df3["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Palavras_Chave)
df3['PALAVRAS-CHAVE-ARTIGO'] = df3['PALAVRAS-CHAVE-ARTIGO'].apply(Separar_Palavras_Chave)
df3['COAUTORES'] = df3['PRODUCAO-BIBLIOGRAFICA'].apply(Unest_Coautores)


df3.drop(labels=['PRODUCAO-BIBLIOGRAFICA','DADOS-GERAIS'],axis=1,inplace=True)

In [77]:
df3.to_csv('views/papers.csv', sep=';', encoding='latin-1')

<br>

## Collecting 'Research Projects Description'

In [106]:
unwind4 = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO.PARTICIPACAO-EM-PROJETO.PROJETO-DE-PESQUISA.@NOME-DO-PROJETO': 1,
                                       }},
                            {'$unwind': '$DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO'},
                          ]))

In [107]:
df4 = pd.DataFrame(unwind4[0])

In [108]:
df.to_csv('views/Research_projects.csv', sep=';', encoding='latin-1')